In [227]:
import time
import re
import pandas as pd
import numpy as np

In [228]:
from bs4 import BeautifulSoup

# Load the html content from the txt file
file_path = r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\txt files\manual_webscraper_html.txt"

with open (file_path, 'r', encoding = 'utf-8') as file:
    file_content = file.read()

soup = BeautifulSoup(file_content, 'html.parser')


In [229]:
print(soup.prettify())

<html class="notranslate" lang="en" translate="no">
 <head>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <link as="image" href="https://avatars.steamstatic.com/c1e1001c4fea91d351c32b183e6ea4c548c79cf1_full.jpg" rel="preload"/>
  <link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-d365fdc27a71b21f.js" rel="preload"/>
  <script async="" defer="" src="https://matomo.stratz.com/matomo.js" type="text/javascript">
  </script>
  <script async="" src="/_next/static/chunks/fd9d1056-e4ed69b157b2b858.js">
  </script>
  <script async="" src="/_next/static/chunks/6099-f30e1c3ea61aa395.js">
  </script>
  <script async="" src="/_next/static/chunks/312-31031cd75562af1b.js">
  </script>
  <script async="" src="/_next/static/chunks/main-app-7077d2f4087aacde.js">
  </script>
  <link as="script" href="https://polyfill.io/v3/polyfill.min.js?features=Intl.RelativeTimeFormat,Intl.RelativeTimeFo

In [230]:
# Load previous data from csv file
df = pd.read_csv(r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\dota.csv")

In [231]:
print(df.head())

      MatchID        Date          Hero Match Duration      Result  Kills  \
0  7325788277  2023-09-08        Jakiro          42:19  Lost Match      6   
1  7325105873  2023-09-08  Witch Doctor          38:24   Won Match     11   
2  7324300278  2023-09-07        Rubick          40:26  Lost Match      5   
3  7298711135  2023-08-23        Oracle          50:02   Won Match     10   
4  7280229073  2023-08-11  Witch Doctor          42:53   Won Match     11   

   Deaths  Assists  Networth     Team      Enemy_pos1  Enemy_pos2 Enemy_pos3  \
0       8       16   15500.0     Dire     drow_ranger       razor    slardar   
1       5       15   16200.0  Radiant      gyrocopter      jakiro    slardar   
2       8       17   11000.0  Radiant  phantom_lancer  juggernaut  dark_seer   
3      10       20       NaN      NaN             NaN         NaN        NaN   
4      10       16       NaN      NaN             NaN         NaN        NaN   

     Enemy_pos4 Enemy_pos5 Ally_pos1   Ally_pos2       A

In [232]:
# Get MatchID from title

title_tag = soup.find('title').text

matchid = int(title_tag.split(' ')[1])


In [233]:
# Determine which row to add match information

row_index = df.loc[df['MatchID'] == matchid].index         # Take the index of the row with the correct matchid
print(row_index)

Index([3], dtype='int64')


In [234]:
my_hero = df.loc[row_index,'Hero']
my_hero = my_hero.iloc[0].lower().replace(' ','_')

In [235]:
all_hero_tags = soup.find_all('img', class_ = 'sc-1e1uy26-0 VSxZd')

radiant_team_tags = all_hero_tags[:5]
dire_team_tags = all_hero_tags[5:10]

In [236]:
radiant_team = []
dire_team = []

#Extract hero name from the image tag and append to enemey_heroes 
for i in radiant_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     radiant_team.append(hero)    

for i in dire_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     dire_team.append(hero)                           



In [237]:
print((radiant_team))
print((dire_team))

['gyrocopter', 'nevermore', 'chaos_knight', 'venomancer', 'phoenix']
['juggernaut', 'kunkka', 'shredder', 'shadow_shaman', 'oracle']


In [238]:
print(my_hero)

oracle


In [239]:
str(my_hero) in radiant_team

False

In [240]:
if str(my_hero) in radiant_team:
    team = 'Radiant'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = dire_team
    ally_heroes = radiant_team
else:
    team = 'Dire'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = radiant_team
    ally_heroes = dire_team
    

In [241]:
print(team)

Dire


In [242]:
position_number = 1
for enemy_hero, ally_hero in zip(enemy_heroes,ally_heroes) :
    df.loc[row_index,'Enemy_pos' + str(position_number)] = enemy_hero
    if position_number != 5:
        df.loc[row_index,'Ally_pos' + str(position_number)] = ally_hero
    position_number += 1


In [243]:
# Find my user_name tag
user_tag = soup.find('span', text = 'MambaMama')

C:\Users\henry\AppData\Local\Temp\ipykernel_31020\383244598.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  user_tag = soup.find('span', text = 'MambaMama')


In [244]:
# Extract networth from tag after my user_name then convert 'k' to multiply by 1000
networth = user_tag.find_next('div', class_ = 'hitagi__sc-1ah81hi-0 jhafUR').text.replace('k', '')
networth = float(networth) * 1000


In [245]:
# Add networth to row
df.loc[row_index,'Networth'] = networth

In [246]:
df.head()

,MatchID,Date,Hero,Match Duration,Result,Kills,Deaths,Assists,Networth,Team,Enemy_pos1,Enemy_pos2,Enemy_pos3,Enemy_pos4,Enemy_pos5,Ally_pos1,Ally_pos2,Ally_pos3,Ally_pos4
0,7325788277,2023-09-08,Jakiro,42:19,Lost Match,6,8,16,15500.0,Dire,drow_ranger,razor,slardar,treant,warlock,muerta,lone_druid,spirit_breaker,silencer
1,7325105873,2023-09-08,Witch Doctor,38:24,Won Match,11,5,15,16200.0,Radiant,gyrocopter,jakiro,slardar,furion,treant,sven,invoker,night_stalker,vengefulspirit
2,7324300278,2023-09-07,Rubick,40:26,Lost Match,5,8,17,11000.0,Radiant,phantom_lancer,juggernaut,dark_seer,witch_doctor,lion,sven,tinker,bristleback,nyx_assassin
3,7298711135,2023-08-23,Oracle,50:02,Won Match,10,10,20,17600.0,Dire,gyrocopter,nevermore,chaos_knight,venomancer,phoenix,juggernaut,kunkka,shredder,shadow_shaman
4,7280229073,2023-08-11,Witch Doctor,42:53,Won Match,11,10,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
csv_dota2 = 'dota.csv'
df.to_csv(csv_dota2, index=False)